# Detección de ironía con red neuronal

In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import preprocessing

# Lectura de archivos

In [61]:
ironicos = pd.read_csv('ironicos_reduced.txt', header = None, delimiter='\t', names=["Tuits"])
no_ironicos = pd.read_csv('no_ironicos_reduced.txt', header = None, delimiter='\t', names=["Tuits"])

ironicos['Clasificacion'] = "Ironico"
no_ironicos['Clasificacion'] = "No_Ironico"

#Se juntan los tuits ironicos y no ironicos en un solo dataset
data = ironicos.append(no_ironicos)
data = data.reset_index(drop=True)
data = data.reindex(np.random.permutation(data.index))
data = data.reset_index(drop=True)
data

,Tuits,Clasificacion
0,Una etapa manejada por mucho dinero bien ganad...,Ironico
1,¿Qué fue lo primero que se te vino a la cabeza...,No_Ironico
2,Cuando en vez de un movil tienes un reloj de 3...,Ironico
3,Extendieron las primarias del Psuv hasta las 1...,Ironico
4,A Mi Me Encanta Que Sea Bonito Por Face Y Que ...,Ironico
5,@ cuando vienen a mty???,No_Ironico
6,Soy exactamente como me ves. No uso máscaras. ...,No_Ironico
7,Otra vez... Audífonos rotos... D: #VibaBravo #...,Ironico
8,Un día cojonudo para trabajar por la noche... ...,Ironico
9,"Me apasiona este clip,es muy bueno",No_Ironico


# Conversión de filas a vectores


In [62]:
def tuit_Vector(mensaje):
    mensaje = mensaje.split()
    for i in range(len(mensaje)):        
        mensaje[i] = mensaje[i].replace(".","")        
        mensaje[i] = mensaje[i].replace("?","")
        mensaje[i] = mensaje[i].replace("¿","")
        mensaje[i] = mensaje[i].replace("!","")
        mensaje[i] = mensaje[i].replace(",","") 
        mensaje[i] = mensaje[i].replace('"',"") 
        mensaje[i] = mensaje[i].replace(":","")
        mensaje[i] = mensaje[i].replace(";","")
        mensaje[i] = mensaje[i].replace("@","")
        mensaje[i] = mensaje[i].replace("#sarcasmo","")
        if(mensaje[i].startswith('#')):                        
            mensaje[i] = mensaje[i].replace(mensaje[i],"")                       
        mensaje[i] = mensaje[i].lower()   
    return mensaje

#Se eliminan simbolos que no son necesarios
print (data.Tuits.head().apply(tuit_Vector))



#numero de plabras unicas en el conjunto de datos
transformar = CountVectorizer(analyzer=tuit_Vector).fit(data['Tuits'])

#Numero de palabras totales
print (len(transformar.vocabulary_))

0    [una, etapa, manejada, por, mucho, dinero, bie...
1    [qué, fue, lo, primero, que, se, te, vino, a, ...
2    [cuando, en, vez, de, un, movil, tienes, un, r...
3    [extendieron, las, primarias, del, psuv, hasta...
4    [a, mi, me, encanta, que, sea, bonito, por, fa...
Name: Tuits, dtype: object
14271


In [63]:
#Proponemos un ejemplo para analizar
tuit1 = data['Tuits'][0]
print(tuit1)


#ver la ocurrencia de las palabras en un tuit, y ver cuantas
#veces se repite esa palabra en dicho tuit
twt1 = transformar.transform([tuit1])
print (twt1)

#Ver las palabras por los indices mostrados en el paso anterior
print (transformar.get_feature_names()[8614])
print (transformar.get_feature_names()[0])  #Es el espacio en blanco
print (transformar.get_feature_names()[3828])

#Se muestra el id de la palabra "me"
print (transformar.vocabulary_.get("me"))

#Se crean las matrices de palabras ya transformadas
matriz = transformar.fit_transform(data['Tuits'])

matriz


Una etapa manejada por mucho dinero bien ganada y trabajado seguramente. #ironia #lamentable http:\\link
  (0, 0)	2
  (0, 1915)	1
  (0, 4257)	1
  (0, 5299)	1
  (0, 5927)	1
  (0, 6607)	1
  (0, 8106)	1
  (0, 8758)	1
  (0, 10199)	1
  (0, 11622)	1
  (0, 12686)	1
  (0, 12968)	1
  (0, 13602)	1
moje

dedicaron
8307


<5838x14271 sparse matrix of type '<class 'numpy.int64'>'
	with 73568 stored elements in Compressed Sparse Row format>

# Ver las palabras por frecuencia de ocurrencia del termino en los mensajes

In [64]:
#se utiliza la funcion TF-IDF la cual sirve para reflejar la importancia
#de cada palabra en el tuit
tfidf_Trans = TfidfTransformer().fit(matriz)

#Se transforma toda la matriz de una vez con los datos actualizados
matriz_tfidf = tfidf_Trans.transform(matriz)
matriz_tfidf

<5838x14271 sparse matrix of type '<class 'numpy.float64'>'
	with 73568 stored elements in Compressed Sparse Row format>

# Entrenamiento de datos  y pruebas

In [65]:
#Se separan los datos para el entrenamiento y pruebas
y = data["Clasificacion"]

#Se separa el conjiunto de datos en subconjuntos de entrenamiento y pruebas
x_train, x_test, y_train, y_test = train_test_split(matriz_tfidf, y, test_size = 0.33, random_state = 10)

# Se escalan los datos
scaler = StandardScaler(copy=False , with_mean=False, with_std=False)
scaler.fit(x_train)


#Aplicamos la escala a la transformación de los datos:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


In [66]:
#generamos el modelo de perceptron multicapas 
#pasamos como número máximo de iteraciones 
mlp = MLPClassifier(hidden_layer_sizes=(2,2,2),max_iter=400)
mlp.fit(x_train, y_train)

#realizamos las predicciones sobre los tweets de prueba
predictions = mlp.predict(x_test)

In [67]:
#Se muestra la precision del modelo y la matriz de confusion
print ('Precision: ', accuracy_score(y_test, predictions, normalize = True))
print ('Matriz de Confusion\n', confusion_matrix(y_test, predictions))
print ('Filas: Valor Real\nColumnas: Valor Predicho')
print ("\n")
print("Reporte: ")
print(classification_report(y_test, predictions))

Precision:  0.858329008822
Matriz de Confusion
 [[868  77]
 [196 786]]
Filas: Valor Real
Columnas: Valor Predicho


Reporte: 
             precision    recall  f1-score   support

    Ironico       0.82      0.92      0.86       945
 No_Ironico       0.91      0.80      0.85       982

avg / total       0.86      0.86      0.86      1927

